In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle
import os

In [35]:
# 1. Load dữ liệu
data_path = "D:/hust/project_3/music-recommendation/backend/data.csv"
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Data file not found at {data_path}")
data = pd.read_csv(data_path)

In [36]:
# 2. Xác định các cột dữ liệu số để huấn luyện
number_cols = ['valence', 'danceability', 'energy', 'acousticness',
               'instrumentalness', 'liveness', 'loudness',
               'speechiness', 'tempo', 'popularity']

# Kiểm tra dữ liệu
if data.empty:
    raise ValueError("The dataset is empty. Please check the data file.")

In [37]:
# 3. Chuẩn hóa dữ liệu với StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[number_cols])

In [38]:
# 4. Huấn luyện mô hình KMeans với k=9
kmeans = KMeans(n_clusters=9, random_state=42)
kmeans.fit(scaled_data)

KMeans(n_clusters=9, random_state=42)

In [39]:
# 5. Lưu mô hình và scaler với tên file không thay đổi
model_path = "kmeans_model.pkl"
scaler_path = "scaler.pkl"

with open(model_path, "wb") as f:
    pickle.dump(kmeans, f)

with open(scaler_path, "wb") as f:
    pickle.dump(scaler, f)

print("KMeans model and scaler have been trained and saved successfully!")

KMeans model and scaler have been trained and saved successfully!


In [40]:
# 6. Test mô hình với một số bài hát mẫu
def get_mean_vector(song_list, spotify_data):
    """
    Calculate the mean vector for a list of songs based on their names.
    """
    song_vectors = []
    for song in song_list:
        try:
            # Tìm kiếm chỉ dựa trên tên bài hát
            song_data = spotify_data[spotify_data['name'].str.lower() == song['name'].lower()].iloc[0]
            song_vector = song_data[number_cols].values
            song_vectors.append(song_vector)
        except IndexError:
            print(f"Warning: Song '{song['name']}' not found in the data.")
    if not song_vectors:
        raise ValueError("None of the provided songs were found in the dataset.")
    return np.mean(np.array(song_vectors), axis=0)

def recommend_songs(song_list, spotify_data, model, scaler, n_songs=5):
    """
    Recommend songs based on input songs.
    """
    try:
        # Tính vector trung bình
        song_center = get_mean_vector(song_list, spotify_data)
        
        # Chuyển thành DataFrame với tên cột tương ứng
        song_center_df = pd.DataFrame([song_center], columns=number_cols)
        
        # Chuẩn hóa vector
        scaled_song_center = scaler.transform(song_center_df)
        scaled_data = scaler.transform(spotify_data[number_cols])
        
        # Tính khoảng cách cosine
        from scipy.spatial.distance import cdist
        distances = cdist(scaled_song_center, scaled_data, 'cosine')
        
        # Lấy chỉ số bài hát gần nhất
        indices = np.argsort(distances[0])[:n_songs]
        
        # Trả về bài hát được đề xuất
        return spotify_data.iloc[indices][['name', 'artists']]
    except ValueError as e:
        print(f"Error in recommendation: {e}")
        return pd.DataFrame()



In [41]:
# Test hàm khuyến nghị
test_songs = [
    {'name': 'Smells Like Teen Spirit'},
    {'name': 'Lithium'},
    {'name': 'All Apologies'}
]

print("\nTesting Recommendation System...")
try:
    recommended_songs = recommend_songs(test_songs, data, kmeans, scaler, n_songs=5)
    if not recommended_songs.empty:
        print("Recommended Songs:")
        print(recommended_songs)
    else:
        print("No recommendations found.")
except Exception as e:
    print(f"Error during recommendation: {e}")


Testing Recommendation System...
Recommended Songs:
                                  name                artists
15217                     Save Tonight   ['Eagle-Eye Cherry']
13809                  Come As You Are            ['Nirvana']
14030                    Runaway Train        ['Soul Asylum']
75137               A Song to Remember  ['A Thousand Horses']
9619   My Sweet Lord - Remastered 2014    ['George Harrison']
